In [2]:
# 1. 필수 라이브러리 설치 및 임포트
# 모듈 설치 (Colab 환경에 mediapipe가 없다면 아래 주석을 풀고 실행하세요.)
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling open

In [1]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from google.colab import drive

# 2. 구글 드라이브 마운트
# 이 코드를 실행하면 구글 인증 절차가 진행됩니다.
drive.mount('/content/drive')

# 3. 동영상 파일 경로 및 추출할 시간 구간 설정
# '목_좌' 데이터에 대한 정보만 담았습니다.
video_path = '/content/drive/MyDrive/2.mp4'
segment_name = 'neck_left'
start_time_ms = 30000
end_time_ms = 44000

Mounted at /content/drive


In [ ]:
# 4. Mediapipe Pose 모델 초기화
mp_pose = mp.solutions.pose
pose = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 5. 동영상 파일 처리 및 데이터 추출
print(f"\n> '{segment_name}' 구간 처리 시작: {start_time_ms/1000}초 ~ {end_time_ms/1000}초")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"오류: '{os.path.basename(video_path)}' 파일을 열 수 없습니다.")
else:
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

    all_landmarks_data = []
    frame_count = 0
    while cap.isOpened():
        if cap.get(cv2.CAP_PROP_POS_MSEC) > end_time_ms:
            break

        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks_row = {}
            # Clearly label landmarks with English names
            landmarks = results.pose_landmarks.landmark
            landmarks_to_extract = {
                'nose': landmarks[0],
                'left_shoulder': landmarks[11],
                'right_shoulder': landmarks[12],
                'left_elbow': landmarks[13],
                'right_elbow': landmarks[14],
                'left_wrist': landmarks[15],
                'right_wrist': landmarks[16],
                'left_hip': landmarks[23],
                'right_hip': landmarks[24]
            }

            for name, landmark in landmarks_to_extract.items():
                landmarks_row[f'{name}_x'] = landmark.x
                landmarks_row[f'{name}_y'] = landmark.y
                landmarks_row[f'{name}_z'] = landmark.z

            landmarks_row['frame'] = frame_count
            all_landmarks_data.append(landmarks_row)

        frame_count += 1

    cap.release()

    if all_landmarks_data:
        df = pd.DataFrame(all_landmarks_data)
        # 6. CSV 파일로 저장
        output_csv_path = f'/content/drive/MyDrive/stretch_pose_data_{segment_name}.csv'
        df.to_csv(output_csv_path, index=False)
        print(f"  > 추출 성공! 추출된 프레임 수: {len(df)}")
        print(f"  > 데이터가 '{output_csv_path}'에 저장되었습니다.")
    else:
        print("  > 오류: 지정된 시간대에 데이터를 추출하지 못했습니다.")

pose.close()

# 특징 공학 (Feature Engineering)

In [ ]:
# 최종 데이터셋 파일을 불러옵니다.
df = pd.read_csv('/content/drive/MyDrive/stretch_pose_data_neck_left.csv')

# --- 1. 거리 계산 (Distance Calculation) ---

# 각 관절 사이의 거리를 계산하는 함수를 정의합니다.
def calculate_distance(df, landmark1_name, landmark2_name):
    return np.sqrt(
        (df[f'{landmark1_name}_x'] - df[f'{landmark2_name}_x'])**2 +
        (df[f'{landmark1_name}_y'] - df[f'{landmark2_name}_y'])**2 +
        (df[f'{landmark1_name}_z'] - df[f'{landmark2_name}_z'])**2
    )

# 1. 왼손목(left_wrist)과 왼쪽 골반(left_hip) 사이의 거리
df['dist_left_wrist_hip'] = calculate_distance(df, 'left_wrist', 'left_hip')

# 2. 오른손목(right_wrist)과 오른쪽 골반(right_hip) 사이의 거리
df['dist_right_wrist_hip'] = calculate_distance(df, 'right_wrist', 'right_hip')

# 3. 양쪽 어깨(left_shoulder, right_shoulder) 사이의 거리
df['dist_shoulder_shoulder'] = calculate_distance(df, 'left_shoulder', 'right_shoulder')

# 4. 코(nose)와 양쪽 골반(hip)의 중간점 사이의 거리
df['dist_nose_hip_mid'] = np.sqrt(
    (df['nose_x'] - (df['left_hip_x'] + df['right_hip_x']) / 2)**2 +
    (df['nose_y'] - (df['left_hip_y'] + df['right_hip_y']) / 2)**2 +
    (df['nose_z'] - (df['left_hip_z'] + df['right_hip_z']) / 2)**2
)

# --- 2. 각도 계산 (Angle Calculation) ---

# 세 개의 점으로 각도를 계산하는 함수를 정의합니다.
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0)) # acos 인자 범위 클리핑

    return np.degrees(angle)

# 1. 왼쪽 팔꿈치 각도 계산 (왼쪽 어깨, 왼쪽 팔꿈치, 왼쪽 손목)
left_elbow_angles = []
for i in range(len(df)):
    angle = calculate_angle(
        (df.loc[i, 'left_shoulder_x':'left_shoulder_z'].values),
        (df.loc[i, 'left_elbow_x':'left_elbow_z'].values),
        (df.loc[i, 'left_wrist_x':'left_wrist_z'].values)
    )
    left_elbow_angles.append(angle)
df['angle_left_elbow'] = left_elbow_angles

# 2. 오른쪽 팔꿈치 각도 계산 (오른쪽 어깨, 오른쪽 팔꿈치, 오른쪽 손목)
right_elbow_angles = []
for i in range(len(df)):
    angle = calculate_angle(
        (df.loc[i, 'right_shoulder_x':'right_shoulder_z'].values),
        (df.loc[i, 'right_elbow_x':'right_elbow_z'].values),
        (df.loc[i, 'right_wrist_x':'right_wrist_z'].values)
    )
    right_elbow_angles.append(angle)
df['angle_right_elbow'] = right_elbow_angles

# --- 최종 파일 저장 ---

# 변경된 데이터프레임을 새로운 파일로 저장합니다.
output_path = '/content/drive/MyDrive/neck_left_processed_features.csv'
df.to_csv(output_path, index=False)
print("\n모든 특징 공학 작업이 완료되었고, '{}' 파일이 저장되었습니다.".format(output_path))

# 최종 데이터셋의 일부를 확인합니다.
print("\n생성된 최종 데이터셋 미리보기:")
print(df.head())


모든 특징 공학 작업이 완료되었고, '/content/drive/MyDrive/neck_left_processed_features.csv' 파일이 저장되었습니다.

생성된 최종 데이터셋 미리보기:
     nose_x    nose_y    nose_z  left_shoulder_x  left_shoulder_y  \
0  0.497206  0.382726 -0.666865         0.551074         0.502677   
1  0.497166  0.382222 -0.495307         0.551080         0.496462   
2  0.497097  0.381935 -0.505793         0.551351         0.491625   
3  0.497530  0.381921 -0.483618         0.551409         0.489397   
4  0.497671  0.381461 -0.441065         0.551621         0.488555   

   left_shoulder_z  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0        -0.396765          0.425542          0.536655         -0.387857   
1        -0.223874          0.425258          0.533818         -0.201757   
2        -0.220789          0.423963          0.533848         -0.216118   
3        -0.209761          0.423235          0.533866         -0.194934   
4        -0.186050          0.423030          0.533895         -0.151688   

   left_elbow_x  

# 데이터 정규화(Data Normalization)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 특징이 추가된 데이터셋 파일을 불러옵니다.
df = pd.read_csv('/content/drive/MyDrive/neck_left_processed_features.csv')

# 정규화할 열들을 선택합니다.
# 'frame' 열은 정규화에서 제외합니다.
columns_to_normalize = df.columns.drop(['frame'])

# MinMaxScaler를 사용하여 데이터 정규화를 수행합니다.
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# 정규화된 데이터셋 미리보기를 출력합니다.
print("정규화된 데이터셋 미리보기:")
print(df[columns_to_normalize].head())

# 정규화된 데이터셋을 새로운 파일로 저장합니다.
df.to_csv('/content/drive/MyDrive/neck_left_normalized.csv', index=False)

print("\n정규화된 데이터가 'neck_left_normalized.csv' 파일로 저장되었습니다.")

정규화된 데이터셋 미리보기:
     nose_x    nose_y    nose_z  left_shoulder_x  left_shoulder_y  \
0  0.032561  0.586660  0.000000         0.000000         0.703944   
1  0.031591  0.571618  0.558483         0.000332         0.477051   
2  0.029913  0.563033  0.524348         0.014039         0.300456   
3  0.040460  0.562634  0.596534         0.016931         0.219122   
4  0.043895  0.548901  0.735062         0.027669         0.188365   

   left_shoulder_z  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0         0.000000          0.656752          1.000000          0.000000   
1         0.494005          0.635948          0.833721          0.619268   
2         0.502822          0.541042          0.835454          0.571482   
3         0.534330          0.487746          0.836534          0.641974   
4         0.602081          0.472735          0.838194          0.785881   

   left_elbow_x  ...  left_hip_z  right_hip_x  right_hip_y  right_hip_z  \
0      0.000000  ...    0.813389     

# 라벨링

In [ ]:
# 정규화된 데이터셋 불러오기
df = pd.read_csv('/content/drive/MyDrive/neck_left_normalized.csv')

# 'label' 열을 추가하고 'neck_left'라는 값으로 채우기
df['label'] = 'neck_left'

# 라벨이 추가된 데이터셋 미리보기
print(df.head())

# 라벨이 추가된 파일을 새로운 이름으로 저장
df.to_csv('/content/drive/MyDrive/neck_left_labeled.csv', index=False)
print("\n'neck_left' 라벨링이 완료되었습니다.")

     nose_x    nose_y    nose_z  left_shoulder_x  left_shoulder_y  \
0  0.032561  0.586660  0.000000         0.000000         0.703944   
1  0.031591  0.571618  0.558483         0.000332         0.477051   
2  0.029913  0.563033  0.524348         0.014039         0.300456   
3  0.040460  0.562634  0.596534         0.016931         0.219122   
4  0.043895  0.548901  0.735062         0.027669         0.188365   

   left_shoulder_z  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0         0.000000          0.656752          1.000000          0.000000   
1         0.494005          0.635948          0.833721          0.619268   
2         0.502822          0.541042          0.835454          0.571482   
3         0.534330          0.487746          0.836534          0.641974   
4         0.602081          0.472735          0.838194          0.785881   

   left_elbow_x  ...  right_hip_y  right_hip_z  frame  dist_left_wrist_hip  \
0      0.000000  ...     0.000000     0.169667    

# 모델링 (오토인코더)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split

# 올바른 동작 데이터만 불러옵니다.
# 'neck_left' 동작 데이터를 오토인코더에 학습시킵니다.
df = pd.read_csv(f'/content/drive/MyDrive/{segment_name}_labeled.csv')

# 'label' 열을 제거하고 입력 특징(features)을 선택합니다.
X = df.drop('label', axis=1).values

# 학습 데이터와 검증 데이터로 나눕니다.
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# ====== 오토인코더 모델 구조 정의 ======
# 입력 레이어
input_layer = Input(shape=(X_train.shape[1],))
# 인코더 (데이터 압축)
encoder = Dense(64, activation='relu')(input_layer)
encoder = Dense(32, activation='relu')(encoder)
encoder = Dense(16, activation='relu')(encoder)
# 디코더 (데이터 복원)
decoder = Dense(32, activation='relu')(encoder)
decoder = Dense(64, activation='relu')(decoder)
decoder = Dense(X_train.shape[1], activation='linear')(decoder)

# 오토인코더 모델 생성
autoencoder = Model(inputs=input_layer, outputs=decoder)

# 모델 컴파일 (오차를 최소화하도록 학습시킵니다)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
history = autoencoder.fit(X_train, X_train,
                          epochs=50,
                          batch_size=256,
                          shuffle=True,
                          validation_data=(X_test, X_test))

# 학습된 모델을 저장합니다.
autoencoder.save(f'/content/drive/MyDrive/{segment_name}_autoencoder_model.h5')

print(f"\n\n모델 학습이 완료되었고, '{segment_name}_autoencoder_model.h5' 파일로 저장되었습니다!")

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 1683.7235 - val_loss: 1615.4622
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 1633.4510 - val_loss: 1560.9088
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 1589.8477 - val_loss: 1497.8257
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 1517.1989 - val_loss: 1425.2766
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 1453.6735 - val_loss: 1342.8079
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 1370.6837 - val_loss: 1249.6923
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 1283.3359 - val_loss: 1146.1047
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 1141.2844 - val_loss: 1036.1511
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 1057.9801 - val_loss: 919.6177
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 937.3140 - val_loss: 800.8897
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 809.1967 - val_loss: 682.9449
Epoch 12/50
2/2 ━━━━━━



모델 학습이 완료되었고, 'neck_left_autoencoder_model.h5' 파일로 저장되었습니다!
